In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Directories for spiral and wave images
spiral_train_dir = r'D:\mini\Parkinson’s Disease Detection Using Spiral Images (Hand Drawings)\Parkinson Dataset\Parkinson Dataset\dataset\spiral'
spiral_test_dir = r'D:\mini\Parkinson’s Disease Detection Using Spiral Images (Hand Drawings)\Parkinson Dataset\Parkinson Dataset\dataset\spiral'

wave_train_dir = r'D:\mini\Parkinson’s Disease Detection Using Spiral Images (Hand Drawings)\Parkinson Dataset\Parkinson Dataset\dataset\wave'
wave_test_dir = r'D:\mini\Parkinson’s Disease Detection Using Spiral Images (Hand Drawings)\Parkinson Dataset\Parkinson Dataset\dataset\wave'

# Function to create data generators
def create_data_generators(train_dir):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2,
        rotation_range=30,  # Increased rotation range
        width_shift_range=0.3,  # Increased shift range
        height_shift_range=0.3,  # Increased shift range
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary',
        subset='training'
    )

    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary',
        subset='validation'
    )
    
    return train_generator, validation_generator

# Function to create and compile the model
def create_resnet50_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    
    # Unfreeze more layers for fine-tuning
    for layer in base_model.layers[-20:]:  # Unfreeze the last 20 layers
        layer.trainable = True
    
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(1024, activation='relu'),  # Increased number of neurons
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.0001),  # Adjusted learning rate
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Function to train the model
def train_model(model, train_generator, validation_generator, model_name):
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=30,  # Increased number of epochs
        steps_per_epoch=len(train_generator),
        validation_steps=len(validation_generator)
    )
    model.save(model_name)
    return model, history

# Function to evaluate the model on test data
def evaluate_model(model, test_dir):
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary'
    )
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
    return test_accuracy

# Train and evaluate for spiral images
spiral_train_gen, spiral_val_gen = create_data_generators(spiral_train_dir)
spiral_model = create_resnet50_model()
trained_spiral_model, spiral_history = train_model(spiral_model, spiral_train_gen, spiral_val_gen, 'spiral_detection_model_resnet50_finetuned.keras')
evaluate_model(trained_spiral_model, spiral_test_dir)

# Train and evaluate for wave images
wave_train_gen, wave_val_gen = create_data_generators(wave_train_dir)
wave_model = create_resnet50_model()
trained_wave_model, wave_history = train_model(wave_model, wave_train_gen, wave_val_gen, 'wave_detection_model_resnet50_finetuned.keras')
evaluate_model(trained_wave_model, wave_test_dir)


Found 82 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - accuracy: 0.6481 - loss: 0.7170 - val_accuracy: 0.7000 - val_loss: 0.6301
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.6972 - loss: 0.7031 - val_accuracy: 0.7000 - val_loss: 0.6142
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.7434 - loss: 0.7034 - val_accuracy: 0.7000 - val_loss: 0.6149
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.7083 - loss: 0.7438 - val_accuracy: 0.7000 - val_loss: 0.6243
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.65

0.7058823704719543